# Homework 2

In [32]:
using Pkg; # The 'Pkg' package is used to install and load other packages
#Pkg.add("PiecewiseLinearOpt")
#Pkg.add("Interpolations")
#Pkg.add("Gurobi")
Pkg.activate("."); Pkg.instantiate(); # The AMO environment is activated and instantiated


#Pkg.status() 'gibt Status zurück

  Activating project at `c:\Users\Leonie Hergeth\Documents\GitHub\AMO_2023\LHe`


In [33]:
using DataStructures # for SortedDict
using DataFrames
using IterTools # for product()
using Plots # for plotting, import and first plot might take a while due to compilation
using JuMP, HiGHS
using Interpolations
import PiecewiseLinearOpt
#import Gurobi

In this part the sub model is developed. It is optimized for 100 different fixed values for X and the objective value is saved in the array alphafunction. This is the optimal value for y dependent on the fixed x.

In [68]:
n = 100
trialValuesX = 0:16/(n-1):16
alphaFunction = [i for i in trialValuesX]

i = 1
for x in trialValuesX
    subModel = Model(HiGHS.Optimizer)

    @variable(subModel, y >=0) 

    @constraint(subModel, y <= 5 + x) 
    @constraint(subModel, y <= 15/2 + x/2) 
    @constraint(subModel, y <= 35/2 - x/2) 
    @constraint(subModel, -y <= 10 - x)

    @objective(subModel, Min, -y)

    optimize!(subModel)
    alphaFunction[i] = objective_value(subModel)
    i += 1
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-1); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -5.0000000000e+00
HiGHS run time      :          0.00
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-1); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     : -5.1616161616e+00
HiGHS run time      :          0.00
Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-4); columns 0(-1); elements 0(-4) - Reduced to emp

Next, the values are interpolated so that two points are connected via a linearized function. This gives us a piece-wise linear function to work with.

It is also tested whether the datapoints are the same.

In [69]:
interp_linear = linear_interpolation(trialValuesX,alphaFunction)

println(alphaFunction[3])
println(interpolatedValues(trialValuesX[3]))

-5.3232323232323235
-5.3232323232323235


This code is used a test function to check whether Julia can optimize with a user-defined function (https://jump.dev/JuMP.jl/stable/manual/nlp/#User-defined-Functions)

In [21]:
function interpolatedValues(x)
    if x > 16
        return 999999999
    elseif x < 0
        return 999999999
    else 
        return interp_linear(x)
    end
end

interpolatedValues (generic function with 1 method)

The master model is defined with its normal part and y is substituted by the piecewise linear function.

In [39]:
masterModel = Model(HiGHS.Optimizer)
@variable(masterModel, 0 <= X <= 16) 

@objective(masterModel, Min, - 0.25 * X + interp_linear(x)) #interpolatedValues(X))

MethodError: MethodError: no method matching (::Interpolations.Extrapolation{Float64, 1, ScaledInterpolation{Float64, 1, Interpolations.BSplineInterpolation{Float64, 1, Vector{Float64}, BSpline{Linear{Throw{OnGrid}}}, Tuple{Base.OneTo{Int64}}}, BSpline{Linear{Throw{OnGrid}}}, Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}}, BSpline{Linear{Throw{OnGrid}}}, Throw{Nothing}})(::VariableRef)
Use square brackets [] for indexing an Array.

Closest candidates are:
  (::Interpolations.Extrapolation{T, N})(!Matched::Number...) where {T, N}
   @ Interpolations C:\Users\Leonie Hergeth\.julia\packages\Interpolations\91PhN\src\extrapolation\extrapolation.jl:45
  (::Interpolations.Extrapolation{T, N})(!Matched::Union{Number, AbstractVector}...) where {T, N}
   @ Interpolations C:\Users\Leonie Hergeth\.julia\packages\Interpolations\91PhN\src\extrapolation\extrapolation.jl:54


Neither the direct interpolated function nor the user-defined function can be optimized by JuMP.

Next, I tried to define a new model which includes binaries to reflect each piece in the objective function. Each piece is represented by a binary linearizator[i], if X is in this piece (meaning smaller or equal than trialValuesX[i], but greater than trialValuesX[i-1]), the linearizator is set to 1. Due to this the objective function can include the sum overall pieces but only the active one is added as the others are multiplied by zero.

This approach does not work as two variables are multiplied by each other and therefore the model is MIQP.

In [67]:
masterModel = Model(HiGHS.Optimizer)
@variable(masterModel, 0 <= X <= 16) 
@variable(masterModel, linearizator[1:n], Bin)

@constraint(masterModel, [i in 2:n], X - trialValuesX[i-1] <= 9999999 * linearizator[i])
@constraint(masterModel, [i in 1:n], trialValuesX[i] - X <= 9999999 * linearizator[i])
#@constraint(masterModel, [i in 2:n], trialValuesX[i-1] * linearizator[i] < X <= trialValuesX[i] * linearizator[i])


@objective(masterModel, Min, 
- 0.25 * X + sum(linearizator[i] * (interpolatedValues(trialValuesX[i]) - interpolatedValues(trialValuesX[i-1])) * (trialValuesX[i] - X) for i in 2:n))

optimize!(masterModel)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
ERROR:   Cannot solve MIQP problems with HiGHS


The easiest solution would be to use Gurobi which already includes a solver with a piecewise formulation. But as I didn't installed it on my computer I couldn't test it.

In [ ]:
import Gurobi
import PiecewiseLinearOpt

model = Model(Gurobi.Optimizer)
@variable(masterModel, 0 <= X <= 16) 
z = PiecewiseLinearOpt.piecewiselinear(model, X, trialValuesX, (alpha) -> interp_linear[alpha])
@objective(model, Min, masterModel, Min, - 0.25 * X + alpha)
optimize!(model)